# Advanced Classification of Disaster-Related Tweets Using Deep Learning

## Introduction
In this project, we will build a deep learning model using Keras to classify tweets as real or fake in the context of disasters. This task is inspired by the "NLP with Disaster Tweets" challenge and enriched with additional data to improve model performance and insights. The dataset provides a fascinating opportunity to explore Natural Language Processing (NLP) techniques on real-world data.

---

## Dataset Overview
### Context
The dataset contains over 11,000 tweets associated with disaster-related keywords such as "crash," "quarantine," and "bush fires." The data structure is based on the original "Disasters on social media" dataset. It includes:
- **Tweets:** The text of the tweet.
- **Keywords:** Specific disaster-related keywords.
- **Location:** The geographical information provided in the tweets.

These tweets were collected on **January 14th, 2020** and cover major events including:
- The eruption of Taal Volcano in Batangas, Philippines.
- The emerging outbreak of **Coronavirus (COVID-19)**.
- The devastating **Bushfires in Australia**.
- The **Iranian downing of flight PS752**.

### Important Note
The dataset contains text that may include profane, vulgar, or offensive language. Please approach with caution during analysis.

---

## Project Goals
### Inspiration
The primary goal of this project is to develop a machine learning model capable of identifying whether a tweet is genuinely related to a disaster or not. This involves:
1. Enriching the already available data with newly collected, manually classified tweets.
2. Leveraging state-of-the-art deep learning methods to extract meaningful insights.
3. Applying NLP techniques to preprocess, clean, and tokenize the tweets for model training.

This notebook will walk through the process of preparing the dataset, building a deep learning model, and evaluating its performance. By the end, we aim to achieve a robust model that can classify disaster tweets with high accuracy.

---

## Why It Matters
Effective classification of disaster-related tweets has numerous practical applications:
- **Emergency Response:** Helps organizations identify critical information in real time.
- **Resource Allocation:** Facilitates better planning by focusing on real disasters.
- **Misinformation Control:** Mitigates the spread of false information during crises.

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
import os

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
# Load the dataset
data = pd.read_csv('tweets.csv')

# Display the first few rows to inspect the dataset
print(data.head())

# Display dataset information (columns, data types, non-null counts)
print(data.info())

   id keyword        location  \
0   0  ablaze             NaN   
1   1  ablaze             NaN   
2   2  ablaze   New York City   
3   3  ablaze  Morgantown, WV   
4   4  ablaze             NaN   

                                                text  target  
0  Communal violence in Bhainsa, Telangana. "Ston...       1  
1  Telangana: Section 144 has been imposed in Bha...       1  
2  Arsonist sets cars ablaze at dealership https:...       1  
3  Arsonist sets cars ablaze at dealership https:...       1  
4  "Lord Jesus, your love brings freedom and pard...       0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11370 entries, 0 to 11369
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        11370 non-null  int64 
 1   keyword   11370 non-null  object
 2   location  7952 non-null   object
 3   text      11370 non-null  object
 4   target    11370 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 444.3+ 

Ensure the dataset contains the required columns, such as:
- `text`: The tweet content.
- `label`: The classification label indicating whether the tweet is fake or not.

In [ ]:
# Verify required columns
assert 'text' in data.columns, "Column 'text' is missing in the dataset."
assert 'target' in data.columns, "Column 'target' is missing in the dataset."

We will split the dataset into training and validation sets using an 80%-20% ratio.

In [ ]:
# Features (tweet content) and labels (fake/true)
X = data['text']       # Features
y = data['target']      # Labels

# Split the dataset (80% training, 20% validation)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Save the training and validation datasets as separate CSV files for later use.

In [ ]:
# Combine features and labels into dataframes
train_df = pd.DataFrame({'text': X_train, 'label': y_train})
test_df = pd.DataFrame({'text': X_test, 'label': y_test})

# Save the dataframes to CSV files
train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv', index=False)

print("Datasets have been saved successfully:")
print("- Training set: train.csv")
print("- Validation set: test.csv")

Datasets have been saved successfully:
- Training set: train.csv
- Validation set: test.csv


## 2. Data Visualization

In this section, we conduct a detailed exploratory data analysis (EDA) to understand the structure and distribution of our dataset. EDA is crucial for identifying potential challenges, trends, and biases within the data, which in turn helps in selecting the most suitable models and preprocessing steps.

### 2.1 Dataset Size

First, we print the size of both the training and testing datasets. This gives us an idea of how many data points we are working with, which is essential when evaluating model performance and understanding the balance of the dataset.


In [ ]:
print("Training dataset size: ", len(X_train))
print("Testing dataset size: ", len(X_test))

## 2.2 Class Distribution in Training Data
Next, we examine the distribution of the target variable in the training set. The target variable indicates whether a tweet is related to a **disaster (1) or not (0)**. Understanding the class distribution helps identify if the dataset is imbalanced, which could influence the choice of model or evaluation metrics (e.g., using precision-recall curves instead of accuracy).

In [ ]:
# Display the count of tweets for each target class in the training set
X_train['target'].value_counts()

We then plot a histogram of the target variable to visualize the distribution of disaster and non-disaster tweets in the training data. This graphical representation makes it easier to spot any imbalance.

In [ ]:
X_train['target'].hist()
plt.ylabel("# tweets")
plt.show()

## 2.3 Exploratory Analysis of Tweet Length
### 2.3.1 Word Count per Tweet
A useful aspect of text data is the length of the text. Here, we explore the number of words per tweet. This metric can help us understand the average tweet size for both disaster-related and non-disaster-related tweets, which may inform decisions on feature engineering, such as tokenization or padding.

We use histograms to display the word count for each category of tweets (disaster vs. non-disaster).

In [ ]:
# Calculate the number of words per tweet for each category (disaster vs non-disaster)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))

tweet_len_0 = X_train[X_train['target'] == 0]['text'].str.split().map(lambda x: len(x))  # Non-disaster tweets
tweet_len_1 = X_train[X_train['target'] == 1]['text'].str.split().map(lambda x: len(x))  # Disaster tweets

ax1.hist(tweet_len_0, color='green')
ax1.set_title('Non-disaster tweets')

ax2.hist(tweet_len_1, color='red')
ax2.set_title('Disaster tweets')

fig.suptitle('Word Count per Tweet')

plt.show()

#### 2.3.2 Unique Word Count per Tweet
Next, we analyze the number of unique words per tweet. This measure indicates how diverse the vocabulary is for each tweet and can be an important factor when building feature sets **like word embeddings or TF-IDF.**

In [ ]:
# Calculate the number of unique words per tweet for each category
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))

tweet_len_0 = X_train[X_train['target'] == 0]['text'].str.split().map(lambda x: len(set(x)))  # Non-disaster tweets
tweet_len_1 = X_train[X_train['target'] == 1]['text'].str.split().map(lambda x: len(set(x)))  # Disaster tweets

ax1.hist(tweet_len_0, color='green')
ax1.set_title('Non-disaster tweets')

ax2.hist(tweet_len_1, color='red')
ax2.set_title('Disaster tweets')

fig.suptitle('Unique Word Count per Tweet')

plt.show()

### 2.3.3 Average Word Length per Tweet
Lastly, we investigate the average length of the words used in the tweets. This measure provides insight into the complexity or simplicity of the language used in disaster vs. non-disaster tweets. A higher average word length might indicate more formal or technical language, whereas shorter words could suggest more informal communication.

In [ ]:
# Calculate the average word length per tweet for each category
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))

tweet_len_0 = X_train[X_train['target'] == 0]['text'].str.split().map(lambda x: np.mean([len(i) for i in x]))  # Non-disaster tweets
tweet_len_1 = X_train[X_train['target'] == 1]['text'].str.split().map(lambda x: np.mean([len(i) for i in x]))  # Disaster tweets

ax1.hist(tweet_len_0, color='green')
ax1.set_title('Non-disaster tweets')

ax2.hist(tweet_len_1, color='red')
ax2.set_title('Disaster tweets')

fig.suptitle('Average Word Length per Tweet')

plt.show()


## 2.4 Further Feature Calculations

In addition to the basic tweet length analysis, we could calculate several other features that may provide additional insights for modeling. These features include:

* Number of **words** at the end of a tweet
* Number of **URLs** per tweet
* **Average** number of characters per tweet
* Number of **characters** per tweet
* Number of punctuation marks per tweet
* Number of **hashtags** per tweet
* Number of **mentions** (@) per tweet

These additional features could be crucial when constructing advanced models or for improving the understanding of tweet content.

## 2.5 Stopwords Analysis

Stopwords are words that do not carry significant meaning by themselves, but help structure or modify other words in a sentence. These include articles, pronouns, prepositions, adverbs, and some verbs. In natural language processing (NLP), stopwords are typically removed because they do not add value to the analysis. For example, search engines like Google do not consider stopwords when indexing content, but they are used when displaying results.

To explore which stopwords are most common in the dataset, we can use the following approach:



In [ ]:
from nltk.corpus import stopwords

stopwords.words('english')

def plot_stopwords(label):
    tweets_stopwords = {}
    for words in X_train[X_train['target'] == label]['text'].str.split():
        sw = list(set(words).intersection(stopwords.words('english')))
        for w in sw:
            if w in tweets_stopwords.keys():
                tweets_stopwords[w] += 1
            else:
                tweets_stopwords[w] = 1

    top = sorted(tweets_stopwords.items(), key=lambda x:x[1],reverse=True)[:10]
    plt.bar(*zip(*top))
    plt.show()

plot_stopwords(0)
plot_stopwords(1)

This will display the 10 most frequent stopwords in disaster and non-disaster tweets separately.

## 2.6 Punctuation Marks Analysis

Next, we analyze the punctuation marks used in the tweets. Punctuation marks can play a role in sentiment analysis and text classification. We examine the most frequently used punctuation marks in both disaster-related and non-disaster-related tweets:

In [ ]:
import string

def plot_punctuation(label):
    tweets_punctuation = {}
    for words in X_train[X_train['target'] == label]['text'].str.split():
        sw = list(set(words).intersection(string.punctuation))
        for w in sw:
            if w in tweets_punctuation.keys():
                tweets_punctuation[w] += 1
            else:
                tweets_punctuation[w] = 1

    top = sorted(tweets_punctuation.items(), key=lambda x:x[1], reverse=True)[:20]
    plt.figure(figsize=(10, 5))
    plt.bar(*zip(*top))
    plt.show()

plot_punctuation(0)
plot_punctuation(1)


This will plot the most common punctuation marks for both types of tweets.

## 2.7 N-grams Analysis

Finally, we perform an analysis of n-grams, which are sequences of n consecutive words. N-grams are useful for capturing patterns in text and can help us understand frequently occurring phrases or expressions.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(ngram_range=(2, 2))
sum_words = cv.fit_transform(X_train['text']).sum(axis=0)

# Calculate frequency of n-grams
words_freq = [(word, sum_words[0, idx]) for word, idx in cv.vocabulary_.items()]
words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)[:20]

plt.figure(figsize=(15, 7))
plt.barh(*zip(*words_freq))
plt.show()

This analysis will highlight the top 20 most frequent 2-grams (bigrams) in the dataset.

These visualizations and additional feature calculations provide deeper insights into the structure and content of the dataset. By understanding features like word count, stopwords, punctuation marks, and n-grams, we can better prepare our data for modeling. Moreover, the analysis helps identify any potential biases or patterns that could influence the performance of the model, ensuring that we approach the task of disaster tweet classification in a well-informed manner.

## 3. Data Cleaning

Data cleaning is a crucial step in preparing a dataset for analysis and machine learning. The goal is to remove unnecessary or problematic elements from the data that could potentially introduce noise or lead to incorrect interpretations. In this section, we will apply several techniques to clean the tweet texts in our dataset.

### 3.1 Removing URLs

One common issue in text data, especially from social media platforms like Twitter, is the presence of URLs. URLs can distract the model and are often irrelevant to the task of tweet classification. Therefore, we define a function to remove URLs from the text using regular expressions.


In [ ]:
import re

def remove_url(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

# Example of removing a URL from a tweet with a different structure
remove_url("Check out this amazing article on machine learning: https://www.example.com/articles/deep-learning-tutorial")

## 3.2 Removing HTML Tags
HTML tags may appear in some tweets, especially when they contain rich text or references to web pages. These tags don't contribute to the meaning of the tweet and should be removed. We can use the `HTMLParser` class to strip HTML tags from the text.

In [ ]:
from html.parser import HTMLParser

class HTMLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs = True
        self.fed = []

    def handle_data(self, d):
        self.fed.append(d)

    def get_data(self):
        return ''.join(self.fed)

def remove_html(text):
    s = HTMLStripper()
    s.feed(text)
    return s.get_data()

# Example of removing HTML tags from a string
remove_html('<tr><td align="left"><a href="../../issues/51/16.html#article">Phrack World News</a></td>')

## 3.3 Removing Emojis

Emojis are often used in tweets to convey emotions or sentiments. While they can be useful for sentiment analysis, they are generally not relevant for a classification task such as disaster prediction. We remove emojis using a regular expression pattern that targets Unicode characters corresponding to emojis.

In [ ]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# Example of removing emojis from a tweet
remove_emoji("Omg another Earthquake 😔😔")

## 3.4 Removing Punctuation
Punctuation marks can sometimes be useful in text analysis, but they often do not add meaningful information for classification tasks. For simplicity, we can remove punctuation from the tweets.

In [ ]:
import string

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

# Example of removing punctuation from a tweet
remove_punctuation("hello #how are you?")

## 3.5 Applying Cleaning Functions to the Dataset
Now that we have defined the functions to remove** URLs, HTML tags, emojis**, and punctuation, we apply them to both the training and testing datasets. This ensures that the data is cleaned before any further processing or model training.

In [ ]:
# Apply cleaning functions to the training set
X_train_prep = X_train.copy()
X_train_prep['text'] = X_train['text'].apply(remove_url)
X_train_prep['text'] = X_train['text'].apply(remove_html)
X_train_prep['text'] = X_train['text'].apply(remove_emoji)
X_train_prep['text'] = X_train['text'].apply(remove_punctuation)

# Apply cleaning functions to the testing set
X_test_prep = X_test.copy()
X_test_prep['text'] = X_test['text'].apply(remove_url)
X_test_prep['text'] = X_test['text'].apply(remove_html)
X_test_prep['text'] = X_test['text'].apply(remove_emoji)
X_test_prep['text'] = X_test['text'].apply(remove_punctuation)

## 3.6 Analyzing N-Grams After Data Cleaning
Once the data has been cleaned, we can proceed with feature extraction. For example, we can analyze n-grams (sequences of words) to see which combinations of words are most frequent in the cleaned training set. Here, we calculate the most frequent bigrams (two-word sequences).

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(ngram_range=(2, 2))
sum_words = cv.fit_transform(X_train_prep['text']).sum(axis=0)

# Calculate frequency of n-grams
words_freq = [(word, sum_words[0, idx]) for word, idx in cv.vocabulary_.items()]
words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)[:20]

# Plot the most frequent bigrams
plt.figure(figsize=(15, 7))
plt.barh(*zip(*words_freq))
plt.show()

Data cleaning is a fundamental step in the text preprocessing pipeline. By removing irrelevant elements such as URLs, HTML tags, emojis, and punctuation marks, we ensure that our model focuses on the meaningful content of the tweets. Once the data is cleaned, we can proceed with further analysis and feature extraction, which will provide the model with cleaner, more relevant input for training.

## 4. Vectorization of the Dataset

In this section, we will transform the textual data into numerical vectors using the **TF-IDF** (Term Frequency-Inverse Document Frequency) method. This is a common technique used in natural language processing (NLP) to convert text data into a form that can be fed into machine learning models. The goal of TF-IDF is to reflect the importance of a word within a document relative to its frequency across all documents.

### 4.1 Preparing Target Labels

Before performing the vectorization, we need to separate the target labels (the classification labels) from the input data (the text of the tweets). The target labels are stored in the `Y_train` variable, which contains the classification for each tweet (0 for non-disaster, 1 for disaster).

In [ ]:
Y_train = X_train_prep['target']

### 4.2 Applying TF-IDF Vectorization

We use the **TfidfVectorizer** from **scikit-learn** to convert the tweet text into numerical vectors. This vectorizer transforms each tweet into a vector where each element represents the importance of a specific word in the tweet.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the training data
X_train = vectorizer.fit_transform(X_train_prep['text'])

# Convert the sparse matrix to a dense array for easier handling
X_train = X_train.toarray()

# Display the transformed training data
X_train

Once the training data is vectorized, we apply the same transformation to the test set, which ensures that both training and testing data are represented in the same vector space.

In [ ]:
# Transform the test data using the fitted vectorizer
X_test = vectorizer.transform(X_test_prep['text'])

# Convert the sparse matrix to a dense array
X_test = X_test.toarray()

Now, both `X_train` and `X_test` contain the vectorized representations of the tweets, which can be used for training machine learning models.

# 5. Data Splitting
In machine learning, it is essential to evaluate the model's performance on data that it hasn't seen during training. To do this, we split the data into training, validation, and testing sets.

## 5.1 Splitting the Data into Training and Validation Sets
We will first split the dataset into training and validation sets. This is done to evaluate the model's performance on data that it hasn't been trained on but still comes from the same distribution.

We use the train_test_split function from scikit-learn to perform this split, reserving 15% of the data for validation.

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets (15% for validation)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.15)

# Print the sizes of the resulting datasets
print("Training subset length: ", len(X_train))
print("Validation subset length: ", len(X_val))
print("Test subset length: ", len(X_test))

## 5.2 Dataset Summary
After splitting the data, we print the lengths of the training, validation, and testing subsets to ensure that the split was performed correctly. The training data will be used to train the model, the validation data will be used for hyperparameter tuning and performance evaluation during training, and the test data will be used to evaluate the final model's performance.